In [1]:
# autoreload
%load_ext autoreload
%autoreload 2


In [40]:
from mldec.datasets.reps_exp_rep_code_data import sample_dataset
n_data = 10
dataset_config = {
    "code_size": 5,
    "repetitions": 5,
    "beta": 1
}
data_tr, triv_tr, _, _  = sample_dataset(n_data, dataset_config, None, seed=None)

(9, 5, 4)
[[array([[1., 1.],
       [2., 1.],
       [2., 2.]], dtype=float32), array([[0, 0, 1, 1, 2, 2],
       [1, 2, 0, 2, 0, 1]], dtype=int64), array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32), array([[0.]], dtype=float32)], [array([[0., 1.],
       [3., 2.],
       [3., 3.]], dtype=float32), array([[0, 0, 1, 1, 2, 2],
       [1, 2, 0, 2, 0, 1]], dtype=int64), array([[0.11111111],
       [0.11111111],
       [0.11111111],
       [1.        ],
       [0.11111111],
       [1.        ]], dtype=float32), array([[0.]], dtype=float32)], [array([[0., 4.]], dtype=float32), array([], shape=(2, 0), dtype=int64), array([], shape=(0, 1), dtype=float32), array([[0.]], dtype=float32)], [array([[1., 1.],
       [1., 2.]], dtype=float32), array([[0, 1],
       [1, 0]], dtype=int64), array([[1.],
       [1.]], dtype=float32), array([[0.]], dtype=float32)], [array([[1., 3.],
       [1., 4.]], dtype=float32), array([[0, 1],
       [1, 0]], dtype=int64), a

In [36]:
data_tr

[Data(x=[3, 2], edge_index=[2, 6], edge_attr=[6, 1], y=[1, 1]),
 Data(x=[3, 2], edge_index=[2, 6], edge_attr=[6, 1], y=[1, 1]),
 Data(x=[1, 2], edge_index=[2, 0], edge_attr=[0, 1], y=[1, 1]),
 Data(x=[2, 2], edge_index=[2, 2], edge_attr=[2, 1], y=[1, 1]),
 Data(x=[2, 2], edge_index=[2, 2], edge_attr=[2, 1], y=[1, 1]),
 Data(x=[3, 2], edge_index=[2, 6], edge_attr=[6, 1], y=[1, 1]),
 Data(x=[6, 2], edge_index=[2, 30], edge_attr=[30, 1], y=[1, 1]),
 Data(x=[3, 2], edge_index=[2, 6], edge_attr=[6, 1], y=[1, 1]),
 Data(x=[4, 2], edge_index=[2, 12], edge_attr=[12, 1], y=[1, 1])]

In [5]:
from mldec.datasets.reps_toric_code_data import *

dataset_config = {
    "repetitions": 4,
    "code_size": 7,
    "p": 0.05,
    "beta": 1,
}


In [6]:
seed = None
n_data = 10
repetitions = dataset_config.get("repetitions") # "cycles" of measurement
code_size = dataset_config.get("code_size")
sampler, detector_coordinates, _ = make_sampler(dataset_config, seed=seed)

# get the surface code grid:
mask = syndrome_mask(code_size, repetitions)
# sample detection events and observable flips
stim_data, observable_flips = sampler.sample(shots=int(n_data), separate_observables=True)
non_empty_indices = (np.sum(stim_data, axis = 1) != 0)
trivial_count = len(observable_flips[~ non_empty_indices])
stim_data = stim_data[non_empty_indices, :]
observable_flips = observable_flips[non_empty_indices]

In [8]:
stim_data.shape

(10, 192)

In [141]:
# Function for creating a single graph as a PyG Data object


In [ ]:
from mldec.utils.graph_representation import *

i = 3


syndrome_3D = stim_to_syndrome_3D(mask, detector_coordinates, stim_data[i])
syndrome_2D = syndrome_3D[0,:,:] # mockup repetition code data for n=5 (syndromes are length 4)
# get defect indices: 
defects, defect_indices_triple = get_node_list_3D(syndrome_3D)
# defects_2D, defect_indices_double = get_node_list_2D(syndrome_2D)



# Use helper function to create node feature matrix as torch.tensor
# (X, Z, N-dist, W-dist, time-dist)
X = get_node_feature_matrix(defects, defect_indices_triple, num_node_features = 5)

# # currently, X_2D is redundant. Single stabilier means that the first axis will always contain '1'...
# # X_2D can just be replaced with np.nonzero(syndrome_2D) which are x_coords, t_coords for all faults.
# # should be 2-D: [d_left, d_time]
# X_2D = get_node_feature_matrix_rep_code(defects_2D, defect_indices_double)


# get the logical equivalence class:
true_eq_class = np.array([int(observable_flips[i])])
# map to graph representation

# first element is 'X', the 5-D [is_x, is_z, d_north, d_west, d_time]
graph_3D = graph_representation.get_3D_graph(syndrome_3D = syndrome_3D,
                    target = true_eq_class,
                    power = 2,
                    m_nearest_nodes = None)

# graph_2D = get_2D_graph(syndrome_2D = syndrome_2D, target=true_eq_class, power=2, m_nearest_nodes=None)




NameError: name 'get_node_list_2D' is not defined

In [41]:
graph_3D

NameError: name 'graph_3D' is not defined

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy.core.multiarray._reconstruct was not an allowed global by default. Please use `torch.serialization.add_safe_globals([_reconstruct])` or the `torch.serialization.safe_globals([_reconstruct])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by 

In [145]:
syndrome_2D

array([[0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=uint8)

In [143]:

graph_2D

[array([[1., 2.],
        [1., 3.],
        [3., 2.],
        [3., 3.],
        [5., 1.],
        [5., 4.]], dtype=float32),
 array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4,
         4, 4, 4, 5, 5, 5, 5, 5],
        [1, 2, 3, 4, 5, 0, 2, 3, 4, 5, 0, 1, 3, 4, 5, 0, 1, 2, 4, 5, 0, 1,
         2, 3, 5, 0, 1, 2, 3, 4]], dtype=int64),
 array([[1.        ],
        [0.25      ],
        [0.25      ],
        [0.0625    ],
        [0.0625    ],
        [1.        ],
        [0.25      ],
        [0.25      ],
        [0.0625    ],
        [0.0625    ],
        [0.25      ],
        [0.25      ],
        [1.        ],
        [0.25      ],
        [0.25      ],
        [0.25      ],
        [0.25      ],
        [1.        ],
        [0.25      ],
        [0.25      ],
        [0.0625    ],
        [0.0625    ],
        [0.25      ],
        [0.25      ],
        [0.11111111],
        [0.0625    ],
        [0.0625    ],
        [0.25      ],
        [0.25      ],
    

In [144]:
X

array([[1., 0., 0., 1., 2.],
       [1., 0., 0., 1., 3.],
       [1., 0., 0., 3., 2.],
       [1., 0., 0., 3., 3.],
       [1., 0., 0., 5., 1.],
       [1., 0., 0., 5., 4.],
       [0., 1., 1., 1., 2.],
       [0., 1., 1., 1., 4.],
       [1., 0., 1., 2., 1.],
       [1., 0., 1., 2., 2.],
       [0., 1., 1., 3., 1.],
       [0., 1., 1., 3., 4.],
       [1., 0., 1., 4., 1.],
       [1., 0., 1., 4., 2.],
       [0., 1., 1., 5., 2.],
       [0., 1., 1., 5., 4.],
       [1., 0., 1., 6., 1.],
       [1., 0., 1., 6., 2.],
       [1., 0., 1., 6., 3.],
       [1., 0., 1., 6., 4.],
       [0., 1., 2., 0., 0.],
       [0., 1., 2., 0., 1.],
       [0., 1., 2., 0., 2.],
       [1., 0., 2., 1., 1.],
       [1., 0., 2., 1., 2.],
       [0., 1., 2., 4., 3.],
       [1., 0., 2., 5., 1.],
       [1., 0., 2., 5., 3.],
       [0., 1., 3., 1., 0.],
       [0., 1., 3., 1., 2.],
       [0., 1., 3., 1., 3.],
       [1., 0., 3., 2., 1.],
       [1., 0., 3., 2., 4.],
       [1., 0., 3., 4., 3.],
       [1., 0.

In [113]:
graph_2D

[array([[1., 1., 1.],
        [1., 1., 3.]], dtype=float32),
 array([[0, 1],
        [1, 0]], dtype=int64),
 array([[0.25],
        [0.25]], dtype=float32),
 array([[0.]], dtype=float32)]

In [ ]:
defects, defect_indices_triple = get_node_list_3D(syndrome_3D)
X = get_node_feature_matrix(defects, defect_indices_triple, num_node_features = 5)
power = 1.

# construct the adjacency matrix!
n_defects = len(defects)
y_coord = defect_indices_triple[0].reshape(n_defects, 1)
x_coord = defect_indices_triple[1].reshape(n_defects, 1)
t_coord = defect_indices_triple[2].reshape(n_defects, 1)

y_dist = np.abs(y_coord.T - y_coord) # yy^T
x_dist = np.abs(x_coord.T - x_coord) # xx^T
t_dist = np.abs(t_coord.T - t_coord) # tt^T
print(y_dist)
print(x_dist)
print(t_dist)


# inverse square of the supremum norm between two nodes
Adj = np.maximum.reduce([y_dist, x_dist, t_dist]) # this is a three-way maximum between three 2D arraysd...
print(Adj)
# set diagonal elements to nonzero to circumvent division by zero
np.fill_diagonal(Adj, 1)
# scale the edge weights
Adj = 1./Adj ** power
# set diagonal elements to zero to exclude self loops
np.fill_diagonal(Adj, 0)

# remove all but the m_nearest neighbours
if m_nearest_nodes is not None:
    for ix, row in enumerate(Adj.T):
        # Do not remove edges if a node has (degree <= m)
        if np.count_nonzero(row) <= m_nearest_nodes:
            continue
        # Get indices of all nodes that are not the m nearest
        # Remove these edges by setting elements to 0 in adjacency matrix
        Adj.T[ix, np.argpartition(row,-m_nearest_nodes)[:-m_nearest_nodes]] = 0.

Adj = np.maximum(Adj, Adj.T) # Make sure for each edge i->j there is edge j->i
n_edges = np.count_nonzero(Adj) # Get number of edges

# get the edge indices:
edge_index = np.nonzero(Adj)
edge_attr = Adj[edge_index].reshape(n_edges, 1)
edge_index = np.array(edge_index)

if target is not None:
    y = target.reshape(1, 1)
else:
    y = None

return [X.astype(np.float32), edge_index.astype(np.int64,), edge_attr.astype(np.float32), y.astype(np.float32)]

[[0 0 1 1 1 1 1 1 2 2 2 2 2 2 2 2]
 [0 0 1 1 1 1 1 1 2 2 2 2 2 2 2 2]
 [1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
 [2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0]]
[[0 0 0 0 1 1 2 2 1 1 0 0 1 1 1 1]
 [0 0 0 0 1 1 2 2 1 1 0 0 1 1 1 1]
 [0 0 0 0 1 1 2 2 1 1 0 0 1 1 1 1]
 [0 0 0 0 1 1 2 2 1 1 0 0 1 1 1 1]
 [1 1 1 1 0 0 1 1 2 2 1 1 0 0 0 0]
 [1 1 1 1 0 0 1 1 2 2 1 1 0 0 0 0]
 [2 2 2 2 1 1 0 0 3 3 2 2 1 1 1 1]
 [2 2 2 2 1 1 0 0 3 3 2 2 1 1 1 1]
 [1 1 1 1 2 2 3 3 0 0 1 1 2 2 2 2]
 [1 1 1 1 2 2 3 3 0 0 1 1 2 2 2 2]
 [0 0 0 0 1 1 2 2 1 1 0 0 1 1 1 1]
 [0 0 0 0 1 1 2 2 1 1 0 0 1 1 1 1]
 [1 1 1 1 0 0 1 1 2

ValueError: stop here

In [104]:
graph

[array([[1., 0., 0., 1., 1.],
        [1., 0., 0., 1., 3.],
        [0., 1., 1., 1., 1.],
        [0., 1., 1., 1., 3.],
        [1., 0., 1., 2., 1.],
        [1., 0., 1., 2., 3.],
        [0., 1., 1., 3., 0.],
        [0., 1., 1., 3., 2.],
        [0., 1., 2., 0., 2.],
        [0., 1., 2., 0., 3.],
        [1., 0., 2., 1., 1.],
        [1., 0., 2., 1., 3.],
        [0., 1., 2., 2., 0.],
        [0., 1., 2., 2., 1.],
        [0., 1., 2., 2., 2.],
        [0., 1., 2., 2., 3.]], dtype=float32),
 array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,
          5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,
          6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7, 

In [101]:
syndrome_2D

array([[0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=uint8)

In [99]:
print(X_2D)

[[1. 1. 1.]
 [1. 1. 3.]]


In [97]:
print(X)

[[1. 0. 0. 1. 1.]
 [1. 0. 0. 1. 3.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 3.]
 [1. 0. 1. 2. 1.]
 [1. 0. 1. 2. 3.]
 [0. 1. 1. 3. 0.]
 [0. 1. 1. 3. 2.]
 [0. 1. 2. 0. 2.]
 [0. 1. 2. 0. 3.]
 [1. 0. 2. 1. 1.]
 [1. 0. 2. 1. 3.]
 [0. 1. 2. 2. 0.]
 [0. 1. 2. 2. 1.]
 [0. 1. 2. 2. 2.]
 [0. 1. 2. 2. 3.]]


In [96]:
defect_indices_double

(array([1, 1], dtype=int64), array([1, 3], dtype=int64))

In [82]:
print(defect_indices_triple)
print(defect_indices_double)

(array([0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64), array([1, 1, 1, 1, 2, 2, 3, 3, 0, 0, 1, 1, 2, 2, 2, 2], dtype=int64), array([1, 3, 1, 3, 1, 3, 0, 2, 2, 3, 1, 3, 0, 1, 2, 3], dtype=int64))
(array([1, 1], dtype=int64), array([1, 3], dtype=int64))


In [76]:
defects_2D

array([1, 1], dtype=uint8)

In [74]:
syndrome_2D[:,4]

array([0, 0, 0, 0], dtype=uint8)

In [64]:
defect_indices_triple

(array([0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([1, 1, 1, 1, 2, 2, 3, 3, 0, 0, 1, 1, 2, 2, 2, 2], dtype=int64),
 array([1, 3, 1, 3, 1, 3, 0, 2, 2, 3, 1, 3, 0, 1, 2, 3], dtype=int64))

In [63]:
defects

array([1, 1, 3, 3, 1, 1, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3], dtype=uint8)

In [28]:
defect_indices_triple 
# (rows, columns, timesteps) triple coordinates of all defects
# each 

(array([2, 2], dtype=int64),
 array([1, 1], dtype=int64),
 array([1, 2], dtype=int64))

In [29]:
defects

array([1, 1], dtype=uint8)

In [27]:
for i in range(5):
    print(syndrome_3D[:,:,i])
    print(mask[:,:,i])
    print()

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[0 1 0 0]
 [0 3 1 3]
 [3 1 3 0]
 [0 0 1 0]]

[[0 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 0 0 0]]
[[0 1 0 0]
 [0 3 1 3]
 [3 1 3 0]
 [0 0 1 0]]

[[0 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 0 0 0]]
[[0 1 0 0]
 [0 3 1 3]
 [3 1 3 0]
 [0 0 1 0]]

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[0 1 0 0]
 [0 3 1 3]
 [3 1 3 0]
 [0 0 1 0]]

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[0 1 0 0]
 [0 3 1 3]
 [3 1 3 0]
 [0 0 1 0]]



In [15]:
len(graph)

4

In [14]:
for i in range(5):
    print(graph[i])

[[0. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 3.]
 [0. 1. 2. 2. 3.]]
[[0 0 0 1 1 1 2 2 2 3 3 3]
 [1 2 3 0 2 3 0 1 3 0 1 2]]
[[1.        ]
 [0.11111111]
 [0.11111111]
 [1.        ]
 [0.25      ]
 [0.25      ]
 [0.11111111]
 [0.25      ]
 [1.        ]
 [0.11111111]
 [0.25      ]
 [1.        ]]
[[1.]]


IndexError: list index out of range